In [1]:
import pandas as pd
from fontTools.subset import subset
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules


In [2]:

data = pd.read_csv("Linhac24-25_Sportlogiq.csv")

data['teaminpossession_prev'] = data['teaminpossession'].shift(1)
data['teaminpossession_next']  = data['teaminpossession'].shift(-1)  # 下一行
data['currentpossession_prev'] = data['currentpossession'].shift(1)
data['currentpossession_next']  = data['currentpossession'].shift(-1)  # 下一行
teaminpossession_fill_index = data[(data['teaminpossession'].isna()) & (data['teaminpossession_prev'] == data['teaminpossession_next'])].index
currentpossession_fill_index = data[(data['currentpossession'].isna()) & (data['currentpossession_prev'] == data['currentpossession_next'])].index
data.loc[currentpossession_fill_index, 'currentpossession'] = data['currentpossession_prev']
data.loc[teaminpossession_fill_index,'teaminpossession'] = data['teaminpossession_prev']
data.drop(['currentpossession_prev','currentpossession_next','teaminpossession_prev','teaminpossession_next'], axis=1, inplace=True)
data['eventname'] = data['eventname']+ '_'+data['type']

In [3]:
# 进球的当前回合的所有event

goal_session = data[(data['eventname'].str.contains('pass')) & (data['type'] == 'south')
][['currentpossession','gameid']]

goal_session.drop_duplicates(keep='last')
goal_session['events'] = None
all_session= data[['currentpossession','gameid']]
all_session.drop_duplicates(keep='last',inplace=True)
all_session.dropna(subset=(['currentpossession']), inplace=True)    
all_session['events'] = None

# 进球的当前回合的所有event
goal_events = list()
all_events = list()
def get_goal_events(row, isAll):
    gameid = row['gameid']
    currentpossession = row['currentpossession']
    # 进球的当前回合的所有event
    events = data[(data['gameid'] == gameid) & (data['currentpossession'] == currentpossession)]['eventname'].tolist()
    (all_events if isAll else goal_events).append(events)
    events_str= ', '.join(events)
    return events_str
    

goal_session['events']  = goal_session.apply(get_goal_events, axis=1,args=(False,))



goal_session

/var/folders/h7/lf9qkc3d547fv7jhp3h4qzyc0000gn/T/ipykernel_22341/2076057508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_session.drop_duplicates(keep='last',inplace=True)
/var/folders/h7/lf9qkc3d547fv7jhp3h4qzyc0000gn/T/ipykernel_22341/2076057508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_session.dropna(subset=(['currentpossession']), inplace=True)
/var/folders/h7/lf9qkc3d547fv7jhp3h4qzyc0000gn/T/ipykernel_22341/2076057508.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,currentpossession,gameid,events
4,0.0,72393,"lpr_faceoffcontested, lpr_contested, pass_sout..."
64,12.0,72393,"lpr_error, pass_south, reception_regular, carr..."
85,15.0,72393,"lpr_contested, lpr_faceoffcontested, pass_sout..."
89,15.0,72393,"lpr_contested, lpr_faceoffcontested, pass_sout..."
95,16.0,72393,"lpr_rebound, rebound_none, pass_south, recepti..."
...,...,...,...
541718,499.0,61409,"lpr_none, puckprotection_body, pass_south, rec..."
541743,504.0,61409,"lpr_faceoff, pass_south, reception_regular, pa..."
541749,505.0,61409,"lpr_none, pass_south, reception_regular, puckp..."
541764,507.0,61409,"lpr_faceoff, pass_south, reception_regular, pa..."


In [4]:
all_session['events']  = all_session.apply(get_goal_events, axis=1,args=(True,))
all_session

/var/folders/h7/lf9qkc3d547fv7jhp3h4qzyc0000gn/T/ipykernel_22341/2686222141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_session['events']  = all_session.apply(get_goal_events, axis=1,args=(True,))


,currentpossession,gameid,events
7,0.0,72393,"lpr_faceoffcontested, lpr_contested, pass_sout..."
14,1.0,72393,"lpr_none, puckprotection_body, carry_none, puc..."
21,2.0,72393,"lpr_contested, lpr_contested, controlledentrya..."
26,3.0,72393,"lpr_none, carry_none, controlledexit_carry, ca..."
32,4.0,72393,"lpr_none, pass_northoffboards, reception_regul..."
...,...,...,...
541790,508.0,61409,"lpr_faceoff, pass_slot, reception_regular, pas..."
541792,509.0,61409,"lpr_none, dumpout_boards"
541795,510.0,61409,"lpr_error, shot_outsideblocked, block_shot"
541799,511.0,61409,"lpr_contested, lpr_contested, shot_slotblocked..."


In [5]:
# 编码数据
te = TransactionEncoder()
te_ary = te.fit(goal_events).transform(goal_events)
df = pd.DataFrame(te_ary, columns=te.columns_)

# 使用 FP-Growth 找频繁项集（支持度最小为 0.3）
frequent_itemsets = fpgrowth(df, min_support=0.1, use_colnames=True)

# 输出频繁项集
print("频繁战术组合（事件集）:")
frequent_itemsets

频繁战术组合（事件集）:


,support,itemsets
0,1.000000,(pass_south)
1,0.937837,(reception_regular)
2,0.338365,(pass_north)
3,0.209279,(lpr_contested)
4,0.183365,(block_pass)
...,...,...
94,0.111202,"(pass_south, block_shot)"
95,0.111202,"(pass_south, block_shot, reception_regular)"
96,0.102837,"(shot_outsideblocked, reception_regular)"
97,0.102837,"(shot_outsideblocked, pass_south)"


In [6]:
# # 编码数据
# te = TransactionEncoder()
# te_ary = te.fit(goal_events).transform(goal_events)
# df = pd.DataFrame(te_ary, columns=te.columns_)
# 
# # 使用 FP-Growth 找频繁项集（支持度最小为 0.3）
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
# # 输出频繁项集
# print("频繁战术组合（事件集）:")
# frequent_itemsets

In [7]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.3)
print("\n关联规则（战术路径推断）:")
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


关联规则（战术路径推断）:


/opt/anaconda3/envs/pylecture/lib/python3.11/site-packages/mlxtend/frequent_patterns/association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


,antecedents,consequents,support,confidence,lift
0,(pass_south),(reception_regular),0.937837,0.937837,1.000000
1,(reception_regular),(pass_south),0.937837,1.000000,1.000000
2,(pass_north),(reception_regular),0.338365,1.000000,1.066284
3,(reception_regular),(pass_north),0.338365,0.360794,1.066284
4,(pass_south),(pass_north),0.338365,0.338365,1.000000
...,...,...,...,...,...
363,"(shot_outsideblocked, reception_regular)",(pass_south),0.102837,1.000000,1.000000
364,"(pass_south, reception_regular)",(shot_outsideblocked),0.102837,0.109653,1.066284
365,(shot_outsideblocked),"(pass_south, reception_regular)",0.102837,1.000000,1.066284
366,(pass_south),"(shot_outsideblocked, reception_regular)",0.102837,0.102837,1.000000


In [11]:
import pandas as pd
from prefixspan import PrefixSpan
from collections import defaultdict

# 示例数据
data = goal_events


# 设置参数
# 初始化 PrefixSpan
ps = PrefixSpan(data)

# 设置最小支持度，找出支持度 >= 2 的频繁序列
ps.minlen = 2  # 最小模式长度
ps.maxlen = 4  # 最大模式长度

# 获取频繁子序列（返回 (支持度, 模式)）
frequent_patterns = ps.frequent(500)

# 输出结果
print("频繁序列模式（支持度 >= 2）:")
for support, pattern in frequent_patterns:
    if pattern[0] != 'pass_south':
        continue
    if support < 500:
        continue
    print(f"支持度: {support}，模式: {pattern}")

频繁序列模式（支持度 >= 2）:
支持度: 18826，模式: ['pass_south', 'reception_regular']
支持度: 5633，模式: ['pass_south', 'reception_regular', 'pass_north']
支持度: 1188，模式: ['pass_south', 'reception_regular', 'pass_north', 'block_pass']
支持度: 5060，模式: ['pass_south', 'reception_regular', 'pass_north', 'reception_regular']
支持度: 2954，模式: ['pass_south', 'reception_regular', 'pass_north', 'pass_south']
支持度: 1209，模式: ['pass_south', 'reception_regular', 'pass_north', 'shot_outside']
支持度: 1664，模式: ['pass_south', 'reception_regular', 'pass_north', 'pass_slot']
支持度: 1469，模式: ['pass_south', 'reception_regular', 'pass_north', 'pass_eastwest']
支持度: 1876，模式: ['pass_south', 'reception_regular', 'pass_north', 'pass_north']
支持度: 639，模式: ['pass_south', 'reception_regular', 'pass_north', 'puckprotection_deke']
支持度: 886，模式: ['pass_south', 'reception_regular', 'pass_north', 'shot_slot']
支持度: 683，模式: ['pass_south', 'reception_regular', 'pass_north', 'block_shot']
支持度: 539，模式: ['pass_south', 'reception_regular', 'pass_north', 'shot_ou

In [23]:
from collections import defaultdict

# 支持度表：{tuple(pattern): support}
support_dict = {tuple(p): s for s, p in frequent_patterns}

# 存放规则：(前件, 后件) -> (支持度, 置信度)
rules = []

for pattern_tuple, sup_full in support_dict.items():
    for i in range(1, len(pattern_tuple)):
        A = pattern_tuple[:i]
        B = pattern_tuple[i:]
        if A in support_dict:
            conf = sup_full / support_dict[A]
            rules.append((A, B, sup_full, conf))

# 输出
print("\n挖掘出的规则（支持度 >= 1000）:")
for A, B, sup, conf in sorted(rules, key=lambda x: -x[3]):

    if all("reception" in item for item in A):
        continue
    if all("reception" in item for item in B):
        continue
    if 'reception' not in A[0]:
        continue
    if conf < 0.5:
        continue

    print(f"{list(A)} → {list(B)} | 支持度: {sup}, 置信度: {conf:.2f}")



挖掘出的规则（支持度 >= 1000）:
['reception_regular', 'pass_outlet', 'controlledexit_carrywithplay'] → ['carry_none'] | 支持度: 639, 置信度: 0.84
['reception_regular', 'pass_stretch', 'reception_regular'] → ['controlledexit_passwithplay'] | 支持度: 811, 置信度: 0.83
['reception_regular', 'pass_eastwest', 'shot_outsideblocked'] → ['block_shot'] | 支持度: 592, 置信度: 0.82
['reception_regular', 'controlledexit_carrywithplay'] → ['carry_none'] | 支持度: 1966, 置信度: 0.82
['reception_regular', 'pass_south', 'controlledexit_carrywithplay'] → ['carry_none'] | 支持度: 641, 置信度: 0.82
['reception_regular', 'reception_regular', 'controlledexit_carrywithplay'] → ['carry_none'] | 支持度: 1390, 置信度: 0.82
['reception_regular', 'pass_north', 'shot_outsideblocked'] → ['block_shot'] | 支持度: 520, 置信度: 0.81
['reception_regular', 'pass_d2d', 'controlledexit_carrywithplay'] → ['carry_none'] | 支持度: 671, 置信度: 0.81
['reception_regular', 'carry_none', 'controlledexit_carrywithplay'] → ['carry_none'] | 支持度: 1823, 置信度: 0.81
['reception_regular', 'pass